In [1]:
## Run the TF-target network analysis acrosss patterning center subtypes
## Overlay the signaling pathway information on TF-target network
source("./nhpf.fun.R")
source("./net.fun.v3.R")
suppressPackageStartupMessages(library(ggforce))
suppressPackageStartupMessages(library(scatterpie))
suppressPackageStartupMessages(library(qgraph))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tidyr’


The following objects are masked from ‘package:Matrix’:

    expand, pack, unpack



Attaching package: ‘ggpubr’


The following object is masked from ‘package:cowplot’:

    get_legend


Loading required package: viridisLite

Attaching SeuratObject


Attaching package: ‘igraph’


The following object is masked from ‘package:tidyr’:

    crossing


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following object is masked from ‘package:tibble’:

    as_data_frame


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union




In [2]:
## system("cp ../MF_pat_v3/load_files/PAT_inte.organizer.inte.rds ./load_files/")
pat <- readRDS(file = "../overview/load_files/PAT_inte.organizer.inte.rds")
sel_cls <- c("PC FGF17", "PC NKX2-1 NKX6-2", "PC NKX2-1 LMO1", "GE_RG_NKX2-1_DLK1", "GE_RG_NKX2-1_OLIG1", 
             "PC RSPO3", "PC TTR", "PC SFRP2", "PC TCF7L2")

In [3]:
## In this cell, cell markers & motif enrichment are calculated
## Use the organizer markers for the TF-target network analysis
## system("cp ../MF_pat_v3/load_files/PAT_markers.Rdata ./load_files/")
load(file = paste0("./load_files/PAT_markers.Rdata"))
cc_genes <- get_genes(input_genes = rownames(pat), gene_type = "cc", revised = TRUE) ## remove cycle genes


## Do some filtering to get markers
slim_res <- lapply(sel_cls, function(cls) {
    ncls <- gsub("PC NKX2-1", "AV NKX2-1", cls) %>%
    gsub("GE_RG_NKX2-1_DLK1", "GE RG NKX2-1 DLK1", .) %>%
    gsub("GE_RG_NKX2-1_OLIG1", "GE RG NKX2-1 OLIG1", .)

    yy <- mar_res[[cls]] %>%
            subset(ratio_fc >= 1.1) %>%
            subset(pct.1 >= 0.1) %>%
            subset(!gene %in% cc_genes) %>%
            mutate(p_val_adj = p.adjust(p_val, method = "fdr")) %>%
            subset(p_val_adj <= 0.01) %>%
            arrange(desc(ratio_fc)) %>%
            ungroup()
    yy
    }) %>%
    setNames(., sel_cls)


## Use motif enrichment to build initial regulons
all_genes <- lapply(slim_res, function(x) x$gene) %>% 
                    unlist() %>% 
                    unique()
step1_file <- paste0("./load_files/PAT_TF-target_motif_res.rds")
if (!file.exists(step1_file)) {
    source("./motif.fun.R")
    mot_res <- MotifEnrich(input_genes = all_genes, input_dir = "./load_files/", 
                           upstream = 800, downstream = 100, nCores = 12)
    saveRDS(mot_res, file = step1_file)
} else {
    mot_res <- readRDS(file = step1_file)
}


In [4]:
names(mot_res)
dim(mot_res$data)
length(all_genes)
head(mot_res$meta)

[1] "data"  "meta"  "score"

[1] 3046  181

[1] 3056

,gene
,<chr>
ARID5B-M00143_1.94d,ARID5B
ARX-M05566_1.94d,ARX
ARX-M08472_1.94d,ARX
ASCL1-M08474_1.94d,ASCL1
ATF3-M08485_1.94d,ATF3
ATF3-M08483_1.94d,ATF3


In [5]:
## Markers used for the TF-target analysis
mars <- lapply(names(slim_res), function(cls) {
    x <- slim_res[[cls]]$gene
    if (cls %in% c("PC NKX2-1 NKX6-2", "PC NKX2-1 LMO1", "GE_RG_NKX2-1_DLK1", "GE_RG_NKX2-1_OLIG1")) {
        y <- x[1:min(200, length(x))]
    } else {
        y <- x[1:min(200, length(x))]
    }
    return(y)
    }) %>%
    setNames(., names(slim_res))
mar_genes <- unlist(mars) %>% unique() %>% 
                intersect(., rownames(mot_res$data))
mars_merge <- mars[c("PC FGF17", "PC RSPO3", "PC TTR", "PC SFRP2", "PC TCF7L2")]
mars_merge[["PC NKX2-1"]] <- union(mars[["PC NKX2-1 NKX6-2"]], mars[["PC NKX2-1 LMO1"]])
mars_merge[["GE NKX2-1"]] <- union(mars[["GE_RG_NKX2-1_DLK1"]], mars[["GE_RG_NKX2-1_OLIG1"]])

sapply(mars_merge, length)

PC FGF17  PC RSPO3    PC TTR  PC SFRP2 PC TCF7L2 PC NKX2-1 GE NKX2-1 
      200       200       200       200       200       355       312

In [6]:
mot_res %>% names()

[1] "data"  "meta"  "score"

In [7]:
## Filter regulons based on co-expression (TF-targets)
mot_res$data <- mot_res$data[mar_genes, ]
mot_res$score <- mot_res$score[mar_genes, ]

raw_regu <- GetInitialRegulons(mot_res = mot_res, pval_thre = 0.05, score_thre = 2.5)

## actual average expression along the UMAP axis
avg_actual <- readRDS(file = paste0("./load_files/", "Pseudobulk_along_UMAP.rds"))

## Filter regulons based on marker genes & co-expression coefficients
regulon_file <- "./load_files/CellType_Regulon_Filtered_20230215.rds"
if (!file.exists(regulon_file)){
    allctp_regu <- lapply(names(mars_merge), function(ctp) {
        print(ctp)
        ## Remove TFs not expressed in the given subclass & filter targets based on markers
        ctp_regu <- raw_regu[names(raw_regu) %in% mars_merge[[ctp]]]
        ctp_regu <- lapply(ctp_regu, function(x) x[names(x) %in% mars_merge[[ctp]]])
        ctp_regu <- ctp_regu[sapply(ctp_regu, length) > 1]

        ## Permuted average expression
        avg_perm <- readRDS(file = paste0("./load_files/Pseudobulk_permuted_along_UMAP_", ctp, ".rds"))

        ## distribution(permuted & actual), (sum(permuted > actual) + 1)/(npermutations + 1)
        ctp_regu <- FilterRegulon_Correlation(regulon = ctp_regu, avg = avg_actual[[ctp]], perm_avg = avg_perm, 
                                              cor_p_thre = 1.3, cor_thre = 0.3)
        ctp_regu <- ctp_regu[sapply(ctp_regu, length) > 1] ## Remove regulons with length of 1
        return(ctp_regu)
        }) %>%
        setNames(., names(mars_merge))
    saveRDS(allctp_regu, file = regulon_file)
}



In [8]:
## Network visualization preparation
allctp_regu <- readRDS(file = regulon_file)


## Only show the subset of network with pathway annotations
path_genes <- readRDS("../overview/load_files/Pathway_updated_20221219.rds") %>% unlist() %>% unique()
tf_kps <- lapply(allctp_regu, function(x) names(x)[sapply(x, function(mm) sum(names(mm) %in% path_genes)) > 0]) %>% 
                unlist() %>% unique()

allctp_regu <- lapply(allctp_regu, function(x) x[intersect(names(x), tf_kps)])


## Transform to network object
netpre_res <- PrepareNet(regulon = allctp_regu)

`summarise()` has grouped output by 'from'. You can override using the `.groups` argument.


In [9]:
tf_kps

[1] "CREB3L4" "DMRTA1"  "ETV4"    "ETV5"    "FEZF1"   "FEZF2"   "FOS"    
 [8] "GLI3"    "HBP1"    "MXI1"    "MYCN"    "NFIA"    "NFIB"    "SIX3"   
[15] "SP8"     "TGIF1"   "ZFHX4"   "ZIC1"    "ZIC2"    "ZIC3"    "ZIC4"   
[22] "ZIC5"    "ZNF423"  "ZNF662"  "ARX"     "BACH2"   "BCL11B"  "DMRTA2" 
[29] "EMX2"    "HES1"    "HES4"    "HEY2"    "JUNB"    "LEF1"    "NFIX"   
[36] "NR2F1"   "TCF4"    "FOXJ1"   "MEF2C"   "MITF"    "NR4A2"   "GSX2"   
[43] "NR2F2"   "PAX6"    "POU3F2"  "SOX3"    "GLI2"    "LHX9"    "RFX4"   
[50] "SHOX2"   "TCF7L2"  "ASCL1"   "DLX1"    "DLX2"    "FOXG1"   "FOXO1"  
[57] "HES5"    "HEY1"    "LHX2"    "MEIS3"   "NKX2-1"  "NKX6-2"  "OLIG1"  
[64] "OLIG2"   "OTX2"    "PBX3"    "PLAGL1"  "SOX21"   "SOX6"    "TCF12"  
[71] "TCF7L1"  "VAX1"    "ZEB1"    "ZEB2"    "ZNF219"  "BCL11A"  "CREB5"  
[78] "DPF1"    "E2F1"    "ESRRG"   "HES6"    "HMGA1"   "HMGA2"   "MEIS2"  
[85] "PBX1"    "POU2F1"  "POU3F3"  "SOX5"    "SOX8"    "TFDP2"   "ZNF704"

In [10]:
head(netpre_res$meta)

,gene,gtype
,<chr>,<chr>
BACH2,BACH2,TF
CREB3L4,CREB3L4,TF
DMRTA1,DMRTA1,TF
ETV4,ETV4,TF
ETV5,ETV5,TF
FEZF1,FEZF1,TF


In [11]:
## Network Visualization 
source("./net.fun.v3.R")
cls_cols <- paste0(c("#d40063", "#dc59e5", "#0c528c", "#6ba4f4", "#0b9376", "#00d49b", "#9f37fa"), "") %>% 
                setNames(., c("PC FGF17", "PC NKX2-1", "PC RSPO3", "PC TTR", "PC SFRP2", "PC TCF7L2", "GE NKX2-1")) 
            ##"#0c6e8c", 
path_cols <- c(FGF = "#CE8002", NOTCH = "#DBD10B", SHH = "#9B7042", 
               RA = "#F90853", EPH = "#1616EF", WNT = "#28E1FA", BMP = "#04AF10")


## Label genes (1. TFs [regulons containing pathway genes] 2. pathway genes)
label_1 <- lapply(allctp_regu, function(x) names(x)[sapply(x, function(mm) sum(names(mm) %in% path_genes)) > 0]) %>% 
                    unlist() %>% unique()
label_2 <- intersect(netpre_res$meta$gene, path_genes)
label_genes <- union(label_1, label_2)

                                                           
## Network plots (full, visualized by cell types)
edg_test <- read.csv(file = "./load_files/PAT_TFnetwork_highlighted_links.txt", 
                     sep = "\t", stringsAsFactors = FALSE, header = TRUE)
#edg_test <- data.frame(from = c("ZIC1", "ZIC1", "NKX2-1", "ZNF219"), to = c("FGF8", "Biubiu", "ZNF219", "SHH"),
#                     stringsAsFactors = FALSE)
plot_tfnet_signal_final(netpre_object = netpre_res, 
                    vertice_size = c(0.7, 2.5, 5.5),
                    label.scale = c(1.5, 1.5), 
                    plot.scale = 3, 
                    vertice_anno_gset = netpre_res$markers, label_genes = path_genes, 
                    vertice_highlight = path_genes, 
                    vertice_cate_cols = cls_cols, 
                    edge_hightlight = edg_test,
                    edge_width = c(0.1, 0.4),
                    remove_small_vertices = TRUE, remove_TFs = FALSE, 
                    redo_layout = FALSE, 
                    file_name = "PAT_TFnetwork_showSubtypes_full_v4", output_dir = "./report/") 

 [1]    4   10   18   58   93  275  318  372  375  461  512  524  568  580  754
[16]  766  770  811  802  816  827  866  893 1024 1300 1301 1306 1309 1310 1328
[31] 1329 1330 1346 1376 1378 1380 1409 1410 1411 1418 1440 1441  124  165  257
[46]  313  604  670  735  105  158  258  480  521  590  791 1250
character(0)


png 
  2

In [12]:
## Network plots (full, visualized by pathways)
path_vis <- readRDS("../overview/load_files/Pathway_updated_20221219.rds")
path_cols <- c(FGF = "#CE8002", NOTCH = "#DBD10B", SHH = "#9B7042", 
                   RA = "#F90853", EPH = "#1616EF", WNT = "#28E1FA", BMP = "#04AF10")
plot_tfnet_signal_final(netpre_object = netpre_res, 
                    vertice_size = c(0.7, 2.5, 5.5),
                    label.scale = c(1.5, 1.5), 
                    plot.scale = 3, 
                    vertice_anno_gset = path_vis, label_genes = path_genes, 
                    vertice_highlight = path_genes, 
                    vertice_cate_cols = path_cols, 
                    edge_hightlight = edg_test,
                    edge_width = c(0.1, 0.4),
                    remove_small_vertices = TRUE, remove_TFs = FALSE, 
                    redo_layout = FALSE, 
                    file_name = "PAT_TFnetwork_showPathway_full_v4", output_dir = "./report/") 

 [1]    4   10   18   58   93  275  318  372  375  461  512  524  568  580  754
[16]  766  770  811  802  816  827  866  893 1024 1300 1301 1306 1309 1310 1328
[31] 1329 1330 1346 1376 1378 1380 1409 1410 1411 1418 1440 1441  124  165  257
[46]  313  604  670  735  105  158  258  480  521  590  791 1250
character(0)


png 
  2

In [13]:
## Output a table summarizing all TF-targets
gset <- readRDS("../overview/load_files/Pathway_updated_20221219.rds")
netout_res <- readRDS(file = regulon_file) %>%
                PrepareNet(regulon = .)

dim(netout_res$meta)
dim(netout_res$links)

## Only out put the links
lk <- netout_res$links[, c("from", "to")]
tf_gene <- netout_res$meta$gene[netout_res$meta$gtype %in% "TF"]

## Annotate TF
lk$from_is_TF <- ifelse(lk$from %in% tf_gene, 1, 0)
lk$to_is_TF <- ifelse(lk$to %in% tf_gene, 1, 0)

## Annotate cell types
for (ii in names(netout_res$markers)){
    lk[, paste0("from_in_subtype_", ii)] <- ifelse(lk$from %in% netout_res$markers[[ii]], 1, 0)
}
for (ii in names(netout_res$markers)){
    lk[, paste0("to_in_subtype_", ii)] <- ifelse(lk$to %in% netout_res$markers[[ii]], 1, 0)
}


## Annotate pathways
for (ii in names(gset)){
    lk[, paste0("from_in_pathway_", ii)] <- ifelse(lk$from %in% gset[[ii]], 1, 0)
}
for (ii in names(gset)){
    lk[, paste0("to_in_pathway_", ii)] <- ifelse(lk$to %in% gset[[ii]], 1, 0)
}

## Check an example
lk %>% filter(from == "NKX2-1")
write.table(lk, file = "./load_files/PAT_TFnetwork_full_table_v4.txt", 
            sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)

`summarise()` has grouped output by 'from'. You can override using the `.groups` argument.


[1] 753   2

[1] 1603    3

from,to,from_is_TF,to_is_TF,from_in_subtype_PC FGF17,from_in_subtype_PC RSPO3,from_in_subtype_PC TTR,from_in_subtype_PC SFRP2,from_in_subtype_PC TCF7L2,from_in_subtype_PC NKX2-1,⋯,from_in_pathway_WNT,from_in_pathway_RA,from_in_pathway_SHH,to_in_pathway_BMP,to_in_pathway_EPH,to_in_pathway_FGF,to_in_pathway_NOTCH,to_in_pathway_WNT,to_in_pathway_RA,to_in_pathway_SHH
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NKX2-1,C1H1orf61,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0
NKX2-1,C6H5orf30,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0
NKX2-1,DNAL4,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0
NKX2-1,FRZB,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,1,0,0
NKX2-1,GPM6A,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0
NKX2-1,LOC106993508,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0
NKX2-1,LOC106999377,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0
NKX2-1,LOC106999572,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0
NKX2-1,LSAMP,1,0,0,0,0,0,0,1,⋯,0,0,0,0,0,0,0,0,0,0


In [14]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux Server 7.9 (Maipo)

Matrix products: default
BLAS/LAPACK: /gpfs/gibbs/pi/sestan.ycga/sm2726/conda_envs/scmultiome/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] qgraph_1.9.3       scatterpie_0.1.8   ggforce_0.4.1      igraph_1.4.0      
 [5] AUCell_1.20.2      SeuratObject_4.1.3 Seurat_4.3.0       viridis_0.6.2     
 [9] viridisLite_0.4.1  ggpubr_0.6.0       cowplot_1.1.1      ggrepel_0.9.3     
[1